   # TP de Méthodes Numériques : 
   # simulation de modèles proies-prédateurs

## 1. Introduction

On souhaite décrire l'évolution de deux populations en interaction: une population de proies
(par exemple des lièvres)
et une population de prédateurs (par exemple des lynx).  On note respectivement $x(t)$ et $y(t)$ les effectifs des
proies et prédateurs 
(normalisés par des valeurs de référence) à l'instant $t$.
On peut modéliser l'évolution de ces populations
à l'aide d'équations différentielles de la forme:
\begin{equation}
\left\{
\begin{array}{lll}
x'(t)&=& x(t) \, f(x(t),y(t),t) \\
y'(t) &=& y(t) \, g(x(t),y(t),t)
\end{array}
\right.
\end{equation}
où $f,g$ sont des fonctions $C^1$ qui décrivent les taux de croissance de chaque population.
La fonction $y\mapsto f(x,y,t)$ est décroissante (terme de prédation) et
$x\mapsto g(x,y,t)$ est croissante.
Par ailleurs, on suppose les fonctions 
$x\mapsto f(x,y,t)$ et $y\mapsto g(x,y,t)$ décroissantes
si on veut prendre en compte une quantité limitée de nourriture.
La dépendance de $f,g$ par rapport au temps permet
de tenir compte des variations saisonnières
de la quantité de nourriture et des taux de natalité.

## 2. Equation de Lotka-Volterra

Il s'agit d'une version très simplifiée du modèle précédent, dans laquelle 
la population de proies dispose de ressources illimitées et 
l'équation différentielle est autonome, i.e. les fonctions $f,g$ ne
dépendent pas de $t$: 
\begin{equation}
\left\{
\begin{array}{lll}
x'(t)&=& x(t) ( 1- y(t) ) \\
y'(t) &=& \alpha\, y(t)(x(t)  - 1).
\end{array}
\right.
\end{equation}
Le paramètre $\alpha$ est strictement positif.
Pour les calculs numériques, on fixera $\alpha =1$.

### 2.1. Analyse du modèle

In [8]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from tqdm import tqdm

In [28]:
def deriv_x_y(x, y, alpha=1):
    xdot = x * (1 - y)
    ydot = alpha * y * (x - 1)
    return xdot, ydot

def test_n_episodes(x_0, y_0, alpha=1, n=1000, verbose=True):
    t = 0
    x = x_0
    y = y_0

    while t < n:
        if t % 1 == 0 and verbose:
            print(f"Episode {t}\t x = {x}, y = {y}")
        
        t += 1
        xdot, ydot = deriv_x_y(x, y, alpha)
        
        if (xdot, ydot) == (0, 0):
            if verbose:
                print(f"Fin de la simulation après {t} episodes. (xdot = {xdot}, ydot = {ydot}) => Point d'équilibre")
                print(f"\nRESULAT:\nx = {x}\ny = {y}")
            return True
        
        x += xdot
        y += ydot
        
        if x > 10**10:
            if verbose:
                print(f"Fin de la simulation après {t} episodes. (x = +inf)")
            break
        if x < 0:
            if verbose:
                print(f"Fin de la simulation après {t} episodes. (x < 0)")
            break
        if y > 10**10:
            if verbose:
                print(f"Fin de la simulation après {t} episodes. (y = +inf)")
            break
        if y < 0:
            if verbose:
                print(f"Fin de la simulation après {t} episodes. (y < 0)")
            break

    if verbose:
        print(f"\nRESULAT:\nx = {x}\ny = {y}")
    return True if t == n else False

def test_condition_initiale(x_0, y_0, alpha, n=1000, verbose=True):
    if test_n_episodes(x_0, y_0, alpha, n, verbose):
        if verbose:
            print("Nous avons trouvé une solution d'équilibre!")
        return True
    else:
        if verbose:
            print("Ces conditions initiales ne donnent pas une solution d'équilibre!")
        return False

def trouver_conditions_initiales(alpha ,x_0=1000, n=1000):
    for y_0 in tqdm(np.linspace(0, 10000, 10000000)):
        if test_condition_initiale(x_0, y_0, alpha, n, verbose=False):
            return y_0
    return -1
alpha = 1

In [89]:
def schema_num(x_0, y_0, alpha, n):
    """ Fonction qui reprend test_n_episodes mais pour un affichage dynamique """
    t = 0
    x = x_0
    y = y_0
    timesteps_x = [x_0]
    timesteps_y = [y_0]
    time = [0]
    
    while t < n:
        t += 1

        xdot, ydot = deriv_x_y(x, y, alpha)
        if (xdot, ydot) == (0, 0):
            break
        
        x += xdot
        y += ydot
        
        if x >= 1 or x < 0 or  y >= 1 or y < 0:
            print(x,y)
            break
        
        time.append(t)
        timesteps_x.append(x)
        timesteps_y.append(y)
    

    fig = plt.figure()
    fig.set_size_inches(16, 8, forward=True)    
    plt.xlim(xmin=0,xmax=len(time)-1)
    
    plt.plot(time, timesteps_x, label='lievre')
    plt.plot(time, timesteps_y, label='lynx')
    
    plt.title("Affichage de notre schema numerique" ,fontsize=16)
    plt.legend()
    plt.show()

    

#### Question 1 
Calculer la solution $(x(t),y(t))$ pour la condition initiale
$(x(0),y(0))=(0,y_0)$ avec $y_0 \geq 0$. Quel est son comportement quand $t \rightarrow +\infty$ ?
Interpréter ce résultat.

In [93]:
widgets.interact(schema_num, 
                 x_0=widgets.fixed(0), 
                 y_0=widgets.IntSlider(value=1,min=0,max=1,step=0.01), 
                 alpha=widgets.FloatSlider(value=1,min=0,max=1,step=0.01),
                 n=widgets.IntSlider(value=200,min=10,max=1000,step=10));

interactive(children=(IntSlider(value=1, description='y_0', max=1, step=0), FloatSlider(value=1.0, description…

#### _Réponse Question 1_ 
Pour la condition initiale $(x(0),y(0))=(0 ,y_0)$ avec $y_0 \geq 0$, nous obtenons la solution: $(x(t), y(t)) \rightarrow(0, 0)$ lorsque $t \rightarrow +\infty$.

Ceci s'explique car les proies, qui commencent à 0, ne peuvent pas se reproduire (ils ne vont pas apparaitre de nulle part), et les prédateurs n'ont rien à manger, et que leur population décroit donc (la vitesse à laquelle elle décroit dépend de $\alpha$).

#### Question 2 
Mêmes questions pour la condition initiale $(x(0),y(0))=(x_0 ,0)$ avec $x_0 \geq 0$. 

In [96]:
widgets.interact(schema_num, 
                 x_0=widgets.FloatSlider(value=1,min=0,max=1,step=0.01), 
                 y_0=widgets.fixed(0), 
                 alpha=widgets.FloatSlider(value=1,min=0,max=1,step=0.01),
                 n=widgets.IntSlider(value=200,min=10,max=1000,step=10));

interactive(children=(FloatSlider(value=1.0, description='x_0', max=1.0, step=0.01), FloatSlider(value=1.0, de…

#### _Réponse Question 2_
Pour la condition initiale $(x(0),y(0))=(x_0, 0)$ avec $x_0 \geq 0$, nous obtenons la solution: $(x(t), y(t)) \rightarrow(+\infty, 0)$ lorsque $t \rightarrow +\infty$.

Ceci s'explique car les proies, qui ne sont pas chassées peuvent se reproduire, et les prédateurs ne peuvent pas augmenter leur population, car celle ce commence à 0.

#### Question 3
Déterminer les solutions d'équilibre, i.e. les solutions $(x,y)$ indépendantes de $t$.

In [97]:
widgets.interact(schema_num, 
                 x_0=widgets.FloatSlider(value=1,min=0,max=1,step=0.01), 
                 y_0=widgets.FloatSlider(value=1,min=0,max=2,step=0.01), 
                 alpha=widgets.FloatSlider(value=1,min=0,max=1,step=0.01),
                 n=widgets.IntSlider(value=200,min=10,max=1000,step=10));

interactive(children=(FloatSlider(value=1.0, description='x_0', max=1.0, step=0.01), FloatSlider(value=1.0, de…

In [98]:
# Premiere solution d'équilibre
print("Test solution 1:")
x_0, y_0 = 0, 10
test_condition_initiale(x_0, y_0, alpha)

print("Test solution 2:")
# Seconde solution d'équilibre
print(trouver_conditions_initiales(alpha, x_0=1 , n=1000))

  1%|          | 51822/10000000 [00:00<00:19, 518087.09it/s]

Test solution 1:
Episode 0	 x = 0, y = 10
Episode 1	 x = 0, y = 0
Fin de la simulation après 2 episodes. (xdot = 0, ydot = 0) => Point d'équilibre

RESULAT:
x = 0
y = 0
Nous avons trouvé une solution d'équilibre!
Test solution 2:


100%|██████████| 10000000/10000000 [00:14<00:00, 691529.98it/s]

-1


#### Question 4
Une fonction $H\, : \, ]0,+\infty[ \times ]0,+\infty[ \rightarrow \mathbb{R}$
est appelée intégrale première de l'équation différentielle
lorsque pour toute solution $(x(t),y(t))$ on a $\frac{d}{dt} H(x(t),y(t))=0$.
Vérifier que 
$$
H(x,y)=\alpha\, (x-\ln{x})+y-\ln{y}
$$
est une intégrale première.

#### Question 5
Tracer différentes courbes de niveau 
$H(x,y)=\mbox{constante}$ (matplotlib.pyplot.contour)
ainsi que la direction du champ de vecteurs $F(x,y)=(\, x ( 1- y )\, ,\, \alpha\, y(x  - 1)\, )$ 
du système (matplotlib.pyplot.quiver) pour $\alpha =1$.

In [25]:
%matplotlib inline
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import numpy as np

#### Question 6
Quand on considère 
des solutions $(x(t),y(t))$
de conditions initiales $(x(0),y(0))$ strictement positives,
que peut-on en déduire pour les trajectoires
$\gamma = \{ (x(t),y(t)),\, t \in \mathbb{R} \} \subset {\left(\mathbb{R}^+ \right)}^2 $
ainsi que pour les fonctions $t \mapsto (x(t),y(t))$ ?

### 2.2. Intégration Numérique

#### 2.2.1. Schéma d'Euler explicite
Afin d'approcher numériquement les solutions du modèle de Lotka-Volterra
pour $t \in [0,t_{\mathrm{max}}]$, 
on discrétise l'équation à l'aide du schéma d'Euler explicite:
\begin{equation}
\left\{
\begin{array}{lll}
x_{k+1}&=& x_k + h\, x_k ( 1- y_k ) \\
y_{k+1} &=& y_k + h\, y_k (x_k  - 1)
\end{array}
\right.
\end{equation}
où $h >0$ est le pas d'intégration en temps et
$(x_k,y_k)$ une approximation de $(x(t_k),y (t_k))$ en $t_k = k\, h$.
On fixe $h=t_{\mathrm{max}}/N$ où $N$ est un entier $\geq 1$.

#### Question 7
Pour $t_{\mathrm{max}}=120$, $(x(0),y(0))=(2,1)$ 
et différentes valeurs du pas de temps $h$, 
tracer les trajectoires 
$(x_k,y_k)_{0 \leq k \leq N} \subset {\left(\mathbb{R}^+ \right)}^2$
ainsi que les graphes de $x_k$, $y_k$ en fonction de $t_k$. 
Qu'observez-vous ? Comparer ces résultats à l'étude théorique 
de la question 6.

#### 2.2.2. Schéma d'Euler implicite
On considère maintenant le schéma d'Euler implicite
\begin{equation}
\left\{
\begin{array}{lll}
x_{k+1}-x_k&=&   h\, x_{k+1} ( 1- y_{k+1} ) \\
y_{k+1}-y_k &=&   h\, y_{k+1} (x_{k+1}  - 1)
\end{array}
\right.
\end{equation}

#### Question 8
Exprimer $x_{k+1}$ en fonction de $x_k$, $y_{k+1}$ et $h$,
puis calculer $y_{k+1}$ en fonction de $(x_k,y_{k},h)$
en résolvant une équation du second degré.

Remarque: lorsque $h=0$ on doit obtenir $(x_{k+1},y_{k+1})=(x_{k},y_{k})$.

#### Question 9
Reprendre la question 7 pour le schéma d'Euler implicite.

#### 2.2.3. Schéma d'Euler semi-implicite
On considère un schéma d'Euler semi-implicite
\begin{equation}
\left\{
\begin{array}{lll}
x_{k+1}-x_k&=&  h\, x_{k} ( 1- y_{k+1} ) \\
y_{k+1} -y_k&=&  h\, y_{k+1} (x_{k}  - 1)
\end{array}
\right.
\end{equation}
dans lequel la variable $x$ est traitée de manière explicite
(évaluée en $t=t_{k}$ dans le second membre de l'équation différentielle)
et $y$ de manière implicite (évaluée en $t=t_{k+1}$).

#### Question 10
Exprimer $(x_{k+1},y_{k+1})$ en fonction de $(x_k,y_{k},h)$.

#### Question 11
Pour $t_{\mathrm{max}}=120$ et différentes valeurs du pas de temps $h$, 
tracer les trajectoires 
$(x_k,y_k)_{0 \leq k \leq N} $
pour différentes conditions initiales $(x(0),y(0))$, 
ainsi que les graphes de $x_k$, $y_k$ en fonction de $t_k$. 
Qu'observez-vous ? Comparer ces résultats à ceux
de la question 6.

## 3. Modèle proies-prédateurs avec croissance limitée
On considère l'équation différentielle:
\begin{equation}
\left\{
\begin{array}{lll}
x'(t)&=& x(t) ( 1- y(t) - c(t)\, x(t)) \\
y'(t) &=&  y(t)(x(t)  - 1)
\end{array}
\right.
\end{equation}
où le coefficient $c(t)>0$ 
tient compte du fait que la population de proies est limitée par la quantité de nourriture.
On suppose que la fonction $c$ est $T$-périodique
(variations saisonnières des ressources) avec $T=12$ (l'unité de temps représente un mois).
Dans ce TP on fixera 
$$
c(t)=\lambda\, \left(1+\frac{1}{2}\, \cos{\left( \frac{\pi}{6}\, t \right)}\right),
$$
où $\lambda >0$ est un paramètre.

On discrétise l'équation différentielle par le
schéma d'Euler semi-implicite
\begin{equation}
\left\{
\begin{array}{lll}
x_{k+1}-x_k&=&  h\, x_{k} ( 1- y_{k+1} - c(t_k)\, x_k) \\
y_{k+1} -y_k&=&  h\, y_{k+1} (x_{k}  - 1)
\end{array}
\right.
\end{equation}

#### Question 12
Exprimer  $(x_{k+1},y_{k+1})$ en fonction de $(t_k,x_k,y_{k},h)$.

#### Question 13
On fixe $t_{\mathrm{max}}=120$ et $h=10^{-3}$.
Pour différentes valeurs du paramètre $\lambda$,
tracer les graphes de $x_k$, $y_k$ en fonction de $t_k$
pour différentes conditions initiales $(x(0),y(0))$. 
Qu'observez-vous ? 
Interpréter le comportement des solutions lorsque
$t\rightarrow +\infty$
en terme d'évolution des populations, ainsi que l'effet du paramètre $\lambda$.